In [70]:
# Base utilities
import os

# Data Mining
import math
import random
random_state = 42
random.seed(random_state)
seed=random_state
import numpy as np 
import pandas as pd
import geopandas as gpd
import osmnx as ox
import pandana as pdn
import pickle as pkl

# Plot
import matplotlib.pyplot as plt
import matplotlib.pyplot as plot
import seaborn as sns

# Learning
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score, train_test_split, KFold

# Models
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC 
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Directories (Si no existen se crean)
for d in ["data", "logs", "models", "networks"]:
    if not os.path.isdir(d):
        os.mkdir(d)

In [71]:
trips = pd.read_csv(f'What-if 2.csv', sep=',')


In [ ]:
#Seleccionar las columna que queremos del fichero
trips = trips[
    ['NIVEL', 'CICLO', 'Centro', 'Promedio', 'SEXO', 'EDAD',
     '¿EN QUÉ MUNICIPIO VIVES?', '¿CÓMO ACOSTUMBRAS A IR DE TU CASA A LA ESCUELA?',
     '¿CUÁNTO TIEMPO HACES DE TU CASA A LA ESCUELA?', '¿CUÁL ES TU SITUACIÓN CONYUGAL?',
     '¿TIENES HIJOS?', 'CÚANTAS PERSONAS, ADEMÁS DE TI, ¿HABITAN EN TU VIVIENDA?',
     '¿CUÁNTOS AUTOMÓVILES O CAMIONETAS TIENEN EN TU HOGAR, INCLUYENDO CAMIONETAS CERRADAS, O CON CABINA O CAJA?',
     'AHORA, AYUDANOS A CONOCER EL MONTO PROMEDIO DE INGRESOS TRIMESTRALES DE TU HOGAR  	Selecciona la opción de monto de ingresos que más se acerque al ingreso trimestral de todos los miembros del hogar',
     '¿DE QUE MATERIAL ES EL PISO DE TU CASA?', '¿TU CASA DISPONE DE AGUA ENTUBADA?',
     'EN TU CASA, ¿DE DÓNDE OBTIENEN LA LUZ ELÉCTRICA?',
     '¿ERES EL PRIMER INTEGRANTE DE TU FAMILIA INMEDIATA (PADRES Y/O HERMANOS) EN ASISTIR A LA UNIVERSIDAD?',
      'Latitud', 'Longitud',
     'Centro específico', 'Latitud_centro', 'Longitud_centro','Nombre de la colonia']
]

print (trips.head().columns)

trips = trips.rename(columns={'¿EN QUÉ MUNICIPIO VIVES?': 'MUNICIPIO'})
trips = trips.rename(columns={'¿CÓMO ACOSTUMBRAS A IR DE TU CASA A LA ESCUELA?': 'Modo transporte'})
trips = trips.rename(columns={'¿CUÁNTO TIEMPO HACES DE TU CASA A LA ESCUELA?': 'Tiempo origen-destino'})
trips = trips.rename(columns={'¿CUÁL ES TU SITUACIÓN CONYUGAL?': 'Situacion conyugal'})
trips = trips.rename(columns={'CÚANTAS PERSONAS, ADEMÁS DE TI, ¿HABITAN EN TU VIVIENDA?': 'Personas en la vivienda'})
trips = trips.rename(columns={'¿CUÁNTOS AUTOMÓVILES O CAMIONETAS TIENEN EN TU HOGAR, INCLUYENDO CAMIONETAS CERRADAS, O CON CABINA O CAJA?': 'Automoviles o camionetas en el hogar incluyendo cabina o caja'})
trips = trips.rename(columns={'AHORA, AYUDANOS A CONOCER EL MONTO PROMEDIO DE INGRESOS TRIMESTRALES DE TU HOGAR  	Selecciona la opción de monto de ingresos que más se acerque al ingreso trimestral de todos los miembros del hogar': 'Ingreso trimestral de todos los miembros del hogar'})
trips = trips.rename(columns={'¿DE QUE MATERIAL ES EL PISO DE TU CASA?': 'Material del piso de la casa'})
trips = trips.rename(columns={'¿TU CASA DISPONE DE AGUA ENTUBADA?': 'Agua entubada en la casa'})
trips = trips.rename(columns={'¿TIENES HIJOS?': 'HIJOS'})
trips = trips.rename(columns={'EN TU CASA, ¿DE DÓNDE OBTIENEN LA LUZ ELÉCTRICA?': 'Luz electrica en la casa'})
trips = trips.rename(columns={'¿ERES EL PRIMER INTEGRANTE DE TU FAMILIA INMEDIATA (PADRES Y/O HERMANOS) EN ASISTIR A LA UNIVERSIDAD?': 'Primer integrante de la familia en asistir a la universidad'})


In [ ]:
#Contar los valores vacíos de cada columna
NaN_mask = trips.apply(lambda col: col.astype(str).str.contains(r"\\N"))
trips = trips.where(~NaN_mask,None)
NaN_mask.sum()

In [74]:
# trips.loc[NaN_mask["previous_trip_purpose"],"previous_trip_purpose"] = trips.loc[NaN_mask["previous_trip_purpose"],"trip_purpose"]
# #Rellenar huecos vacíos con la media de esos valores
# for column in ["trip_taker_"+c for c in ["age","individual_income","household_size","household_income"]]:
#     trips.loc[NaN_mask[column],column] = pd.to_numeric(trips.loc[~NaN_mask[column],column]).mean()
# #Eliminar filas que no tienen lat y long
# for column in ["trip_taker_"+c for c in ["home_bgrp_lat","home_bgrp_lng","work_bgrp_lat","work_bgrp_lng"]]:
#     trips = trips[~NaN_mask[column]] 
# #Si tiene un valor nulo se le pone ese indicado
# trips.loc[NaN_mask["trip_taker_sex"],"trip_taker_sex"] = "male"
# trips.loc[NaN_mask["trip_taker_race_ethnicity"],"trip_taker_race_ethnicity"] = "white_not_hispanic_or_latino"
# trips.loc[NaN_mask["trip_taker_employment_status"],"trip_taker_employment_status"] = "employed"
# trips.loc[NaN_mask["trip_taker_wfh"],"trip_taker_wfh"] = "in_person"
# # trips.loc[NaN_mask["trip_taker_commute_mode"],"trip_taker_commute_mode"] = "private_auto"
# # trips.loc[NaN_mask["trip_taker_available_vehicles"],"trip_taker_available_vehicles"] = "one"
# trips.loc[NaN_mask["trip_taker_resident_type"],"trip_taker_resident_type"] = "core"

## Sampling, filtering and label aggregation

In [ ]:
trips = trips[trips['Modo transporte'] != 'Bike']
for p in [f"{label}: {trips[trips['Modo transporte'] == label].shape[0]:,}" for label in trips['Modo transporte'].unique()]:
    print(p)
print(f"Total: {trips.shape[0]:,}")


In [ ]:
for p in [f"{label}: {trips[trips['Modo transporte'] == label].shape[0]:,}" for label in trips['Modo transporte'].unique()]:
    print(p)
print(f"Total: {trips.shape[0]:,}")

In [ ]:
for p in [f"{label}: {trips[trips['Modo transporte'] == label].shape[0]:,}" for label in trips['Modo transporte'].unique()]:
    print(p)
print(f"Total: {trips.shape[0]:,}")

In [ ]:
#Establecer rangos en los ingresos (de string a numerico)
trips['Ingreso trimestral de todos los miembros del hogar'] = trips['Ingreso trimestral de todos los miembros del hogar'].replace({"$0 a $9,113": 1, "$9,114 a $16,100": 2,"$16,101 a $21,428":3, "$21,429 a $26,696":4,"$26,697 a $32,318":5, "$32,319 a $38,957":6, "$38,958 a $47,264":7, "$47,265 a $58,885":8, "$58,886 a $78,591":9, "$78,592 a $166,750":10 })

print(trips['Ingreso trimestral de todos los miembros del hogar'].unique())


trips['Automoviles o camionetas en el hogar incluyendo cabina o caja'] = trips['Automoviles o camionetas en el hogar incluyendo cabina o caja'].replace({"2 ó más":2,"1":1, "0":0})
print(trips['Automoviles o camionetas en el hogar incluyendo cabina o caja'].unique())

trips['Personas en la vivienda'] = trips['Personas en la vivienda'].replace({"Vivo solo":0,"1":1,"2":2,"3":3,"4":4,"5":5,"6":6,"7":7,"8":8,"9":9,"10":10,"Mas de 10":11 })
print(trips['Personas en la vivienda'].unique())


In [81]:
trips.to_csv(f"data/Guadalajara_no_NaNs.csv")

In [82]:
trips = pd.read_csv(f"data/Guadalajara_no_NaNs.csv",index_col=0)

## Columns Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
import os
import pickle as pkl

# Directorio donde se encuentran los encoders
encoders_directory = "models/encoders"

# Definir las columnas categóricas en 'trips'
cat_cols = ["Centro", "NIVEL", "SEXO", "Situacion conyugal", "MUNICIPIO", "HIJOS", "Material del piso de la casa", "Agua entubada en la casa", "Luz electrica en la casa", "Primer integrante de la familia en asistir a la universidad", "Centro específico"]

# Diccionario para almacenar los encoders
encoders = {}

# Iterar a través de las columnas categóricas
for col in cat_cols:
    # Cargar el encoder desde el archivo pickle
    encoder_path = os.path.join(encoders_directory, f'{col}.pkl')
    with open(encoder_path, 'rb') as file:
        encoder = pkl.load(file)
        encoders[col] = encoder

    # Manejar nuevas categorías: reemplazar con un valor predeterminado (ejemplo: -1)
    trips[col] = trips[col].apply(lambda x: encoder.transform([x])[0] if x in encoder.classes_ else -1)

# Mostrar el DataFrame 'trips' después de la transformación
print(trips)




In [ ]:


# Cargar el archivo Excel
weights_df = pd.read_excel('Weight_what-if-2.xlsx')

# Iterar sobre cada fila del dataframe trips
for index, row in trips.iterrows():
    # Filtrar el dataframe weights_df para encontrar la fila correspondiente
    # donde coinciden los valores de Latitud, Longitud, Latitud_centro y Longitud_centro
    filtered_row = weights_df[(weights_df['Latitud'] == row['Latitud']) & 
                              (weights_df['Longitud'] == row['Longitud']) &
                              (weights_df['Latitud_centro'] == row['Latitud_centro']) &
                              (weights_df['Longitud_centro'] == row['Longitud_centro'])]
    
    # Si hay una fila coincidente, actualizar los valores en el dataframe trips
    if not filtered_row.empty:
        trips.at[index, 'walk_weight'] = filtered_row['walk_weight'].values[0]
        trips.at[index, 'transit_weight'] = filtered_row['transit_weight'].values[0]
        trips.at[index, 'drive_weight'] = filtered_row['drive_weight'].values[0]

print (trips.head().columns)


In [87]:
trips = trips.reset_index(drop=True)

In [88]:
trips = trips.drop(columns=["Nombre de la colonia"],errors='ignore')

## Saving

In [90]:
trips.to_csv(f"data/What-if 2_encoded_OK.csv")
